In [1]:
import requests
from requests.auth import HTTPBasicAuth 
BASE_URL = "http://localhost:2480"
AUTH = HTTPBasicAuth("root", "playwithdata")

In [5]:
def create_database(name):
    url = f"{BASE_URL}/api/v1/server"
    payload = { "command": f"create database {name}" }
    response = requests.post(url, json=payload, auth=AUTH)
    print(f"[CREATE DB] Status: {response.status_code}\nResponse: {response.text}\n")

create_database("LIBRARY_DATA")

[CREATE DB] Status: 200
Response: {"result":"ok"}



In [ ]:
DB_NAME = "LIBRARY_DATA"

In [59]:

def run_sql(sql):
    logging.info(f"Running SQL:\n{sql.strip()}")
    url = f"{BASE_URL}/api/v1/command/{DB_NAME}"
    payload = {"command": sql, "language": "sqlscript"}
    response = requests.post(url, json=payload, auth=AUTH)
    try:
        json_data = response.json()
        if response.status_code == 200:
            logging.info(f" Success: {json_data}")
        else:
            logging.error(f" HTTP {response.status_code}: {json_data}")
        return json_data
    except Exception:
        logging.error(f" Non-JSON Response: {response.text}")

In [34]:


# --- Create Schema ---
def create_schema():
    logging.info("🔧 Creating schema...")
    schema_sql = """
    CREATE VERTEX TYPE Book;
    CREATE VERTEX TYPE Author;
    CREATE VERTEX TYPE Publisher;
    CREATE VERTEX TYPE Category;

    CREATE EDGE TYPE WRITTEN_BY;
    CREATE EDGE TYPE PUBLISHED_BY;
    CREATE EDGE TYPE HAS_CATEGORY;
    """
    run_sql(schema_sql)

# --- Insert Book with Relations ---
def insert_rich_book(book):
    logging.info(f"📚 Inserting book: {book['title']}")
    summary = book["summary"].replace("'", "\\'")
    run_sql(f"""
    INSERT INTO Book SET
        title = '{book["title"]}',
        isbn = '{book["isbn"]}',
        year = {book["year"]},
        summary = '{summary}',
        edition = '{book["edition"]}'
    """)

    pub = book["publisher"]
    run_sql(f"""
    INSERT INTO Publisher SET name = '{pub["name"]}', location = '{pub["location"]}';
    CREATE EDGE PUBLISHED_BY FROM (SELECT FROM Book WHERE isbn = '{book["isbn"]}')
    TO (SELECT FROM Publisher WHERE name = '{pub["name"]}');
    """)

    for author in book["authors"]:
        run_sql(f"""
        INSERT INTO Author SET name = '{author["name"]}', nationality = '{author["nationality"]}', birth_year = {author["birth_year"]};
        CREATE EDGE WRITTEN_BY FROM (SELECT FROM Book WHERE isbn = '{book["isbn"]}')
        TO (SELECT FROM Author WHERE name = '{author["name"]}');
        """)

    for cat in book["categories"]:
        run_sql(f"INSERT INTO Category SET name = '{cat}';")
        run_sql(f"""
        CREATE EDGE HAS_CATEGORY FROM (SELECT FROM Book WHERE isbn = '{book["isbn"]}')
        TO (SELECT FROM Category WHERE name = '{cat}');
        """)

    logging.info(f"✅ Finished inserting book: {book['title']}\n")

# --- Main ---
if __name__ == "__main__":
    create_schema()

    book_data = {
        "title": "A Brief History of Time",
        "isbn": "9780553380163",
        "year": 1988,
        "summary": "A landmark volume in science writing by one of the great minds of our time.",
        "edition": "First",
        "authors": [
            {"name": "Stephen Hawking", "nationality": "British", "birth_year": 1942}
        ],
        "publisher": {"name": "Bantam Books", "location": "New York"},
        "categories": ["Science", "Cosmology"]
    }

    insert_rich_book(book_data)


2025-07-23 13:58:54,588 [INFO] 🔧 Creating schema...
2025-07-23 13:58:54,589 [INFO] Running SQL:
CREATE VERTEX TYPE Book;
    CREATE VERTEX TYPE Author;
    CREATE VERTEX TYPE Publisher;
    CREATE VERTEX TYPE Category;

    CREATE EDGE TYPE WRITTEN_BY;
    CREATE EDGE TYPE PUBLISHED_BY;
    CREATE EDGE TYPE HAS_CATEGORY;
2025-07-23 13:58:54,706 [ERROR] ❌ HTTP 500: {'error': 'Error on transaction commit', 'detail': 'Type Book already exists', 'exception': 'com.arcadedb.exception.CommandExecutionException'}
2025-07-23 13:58:54,707 [INFO] 📚 Inserting book: A Brief History of Time
2025-07-23 13:58:54,708 [INFO] Running SQL:
INSERT INTO Book SET
        title = 'A Brief History of Time',
        isbn = '9780553380163',
        year = 1988,
        summary = 'A landmark volume in science writing by one of the great minds of our time.',
        edition = 'First'
2025-07-23 13:58:54,748 [INFO] ✅ Success: {'result': [{'@rid': '#1:6', '@type': 'Book', '@cat': 'v', 'title': 'A Brief History of Ti

Storing data

In [35]:
more_books = [
    {
        "title": "Sapiens: A Brief History of Humankind",
        "isbn": "9780062316097",
        "year": 2011,
        "summary": "Explores the history and impact of Homo sapiens.",
        "edition": "First",
        "authors":[ {
            "name": "Yuval Noah Harari",
            "nationality": "Israeli",
            "birth_year": 1976
        }],
        "publisher": {
            "name": "Harper",
            "location": "New York"
        },
        "categories": ["History", "Anthropology"]
    },
    {
        "title": "Thinking, Fast and Slow",
        "isbn": "9780374533557",
        "year": 2011,
        "summary": "Daniel Kahneman presents decades of research to help us understand how decisions are made.",
        "edition": "First",
        "authors": [{
            "name": "Daniel Kahneman",
            "nationality": "Israeli-American",
            "birth_year": 1934
        }],
        "publisher": {
            "name": "Farrar, Straus and Giroux",
            "location": "New York"
        },
        "categories": ["Psychology", "Cognitive Science"]
    },
    {
        "title": "To Kill a Mockingbird",
        "isbn": "9780061120084",
        "year": 1960,
        "summary": "A novel about the serious issues of rape and racial inequality.",
        "edition": "First",
        "authors":[ {
            "name": "Harper Lee",
            "nationality": "American",
            "birth_year": 1926
        }],
        "publisher": {
            "name": "J.B. Lippincott & Co.",
            "location": "Philadelphia"
        },
        "categories": ["Fiction", "Social Justice"]
    },
    {
        "title": "1984",
        "isbn": "9780451524935",
        "year": 1949,
        "summary": "A dystopian novel set in a totalitarian society under constant surveillance.",
        "edition": "First",
        "authors": [{
            "name": "George Orwell",
            "nationality": "British",
            "birth_year": 1903
        }],
        "publisher": {
            "name": "Secker & Warburg",
            "location": "London"
        },
        "categories": ["Fiction", "Dystopia", "Politics"]
    },
    {
        "title": "The Origin of Species",
        "isbn": "9781509827695",
        "year": 1859,
        "summary": "The foundational text of evolutionary biology.",
        "edition": "Original",
        "authors": [{
            "name": "Charles Darwin",
            "nationality": "British",
            "birth_year": 1809
        }],
        "publisher": {
            "name": "John Murray",
            "location": "London"
        },
        "categories": ["Biology", "Evolution", "Science"]
    }
]

# Insert all books in the list
for book in more_books:
    insert_rich_book(book)


2025-07-23 14:03:51,995 [INFO] 📚 Inserting book: Sapiens: A Brief History of Humankind
2025-07-23 14:03:52,031 [INFO] Running SQL:
INSERT INTO Book SET
        title = 'Sapiens: A Brief History of Humankind',
        isbn = '9780062316097',
        year = 2011,
        summary = 'Explores the history and impact of Homo sapiens.',
        edition = 'First'
2025-07-23 14:03:59,483 [INFO] ✅ Success: {'result': [{'@rid': '#1:7', '@type': 'Book', '@cat': 'v', 'title': 'Sapiens: A Brief History of Humankind', 'isbn': '9780062316097', 'year': 2011, 'summary': 'Explores the history and impact of Homo sapiens.', 'edition': 'First'}]}
2025-07-23 14:03:59,494 [INFO] Running SQL:
INSERT INTO Publisher SET name = 'Harper', location = 'New York';
    CREATE EDGE PUBLISHED_BY FROM (SELECT FROM Book WHERE isbn = '9780062316097')
    TO (SELECT FROM Publisher WHERE name = 'Harper');
2025-07-23 14:04:01,010 [INFO] ✅ Success: {'result': [{'@rid': '#14:45', '@type': 'PUBLISHED_BY', '@cat': 'e', '@in': '#7

Retrieval

In [37]:
run_sql("SELECT title FROM Book")

2025-07-23 15:46:46,077 [INFO] Running SQL:
SELECT title FROM Book
2025-07-23 15:46:46,991 [INFO] ✅ Success: {'result': [{'title': 'A Brief History of Time'}, {'title': 'A Brief History of Time'}, {'title': 'A Brief History of Time'}, {'title': 'A Brief History of Time'}, {'title': 'Sapiens: A Brief History of Humankind'}, {'title': 'Sapiens: A Brief History of Humankind'}, {'title': 'A Brief History of Time'}, {'title': 'Sapiens: A Brief History of Humankind'}, {'title': 'Thinking, Fast and Slow'}, {'title': 'To Kill a Mockingbird'}, {'title': '1984'}, {'title': 'The Origin of Species'}]}


{'result': [{'title': 'A Brief History of Time'},
  {'title': 'A Brief History of Time'},
  {'title': 'A Brief History of Time'},
  {'title': 'A Brief History of Time'},
  {'title': 'Sapiens: A Brief History of Humankind'},
  {'title': 'Sapiens: A Brief History of Humankind'},
  {'title': 'A Brief History of Time'},
  {'title': 'Sapiens: A Brief History of Humankind'},
  {'title': 'Thinking, Fast and Slow'},
  {'title': 'To Kill a Mockingbird'},
  {'title': '1984'},
  {'title': 'The Origin of Species'}]}

In [43]:
def delete_duplicate_books():
    # Step 1: Get all books by title
    sql = "SELECT title, count(*) as count FROM Book GROUP BY title"
    result = run_sql(sql).get("result", [])

    for row in result:
        title = row["title"]
        count = row["count"]

        if count > 1:
            # Step 2: Get RIDs for books with this title
            rid_result = run_sql(f"SELECT @rid FROM Book WHERE title = '{title}'")
            rids = [r["@rid"] for r in rid_result.get("result", [])]

            # Step 3: Keep one, delete the rest
            for rid in rids[1:]:  # Skip the first
                delete_sql = f"DELETE VERTEX FROM (SELECT FROM Book WHERE @rid = '{rid}')"
                run_sql(delete_sql)

delete_duplicate_books()


2025-07-23 15:59:26,715 [INFO] Running SQL:
SELECT title, count(*) as count FROM Book GROUP BY title
2025-07-23 15:59:27,042 [INFO] ✅ Success: {'result': [{'title': 'A Brief History of Time', 'count': 5}, {'title': 'Sapiens: A Brief History of Humankind', 'count': 3}, {'title': 'Thinking, Fast and Slow', 'count': 1}, {'title': 'To Kill a Mockingbird', 'count': 1}, {'title': '1984', 'count': 1}, {'title': 'The Origin of Species', 'count': 1}]}
2025-07-23 15:59:27,046 [INFO] Running SQL:
SELECT @rid FROM Book WHERE title = 'A Brief History of Time'
2025-07-23 15:59:27,106 [INFO] ✅ Success: {'result': [{'@rid': '#1:0'}, {'@rid': '#1:1'}, {'@rid': '#1:2'}, {'@rid': '#1:3'}, {'@rid': '#1:6'}]}
2025-07-23 15:59:27,108 [INFO] Running SQL:
DELETE VERTEX FROM (SELECT FROM Book WHERE @rid = '#1:1')
2025-07-23 15:59:27,493 [INFO] ✅ Success: {'result': [{'count': 1}]}
2025-07-23 15:59:27,501 [INFO] Running SQL:
DELETE VERTEX FROM (SELECT FROM Book WHERE @rid = '#1:2')
2025-07-23 15:59:27,571 [INFO

In [61]:
sql = """
    SELECT 
      title,
      isbn,
      year,
      summary,
      edition,
      (select expand(out('WRITTEN_BY')) from Book where @rid = b.@rid) as authors,
      (select expand(out('PUBLISHED_BY')) from Book where @rid = b.@rid) as publisher,
      (select expand(out('BELONGS_TO')) from Book where @rid = b.@rid) as categories
    from Book as b;
    """
print(run_sql(sql))

2025-07-23 16:19:17,621 [INFO] Running SQL:
SELECT 
      title,
      isbn,
      year,
      summary,
      edition,
      (select expand(out('WRITTEN_BY')) from Book where @rid = b.@rid) as authors,
      (select expand(out('PUBLISHED_BY')) from Book where @rid = b.@rid) as publisher,
      (select expand(out('BELONGS_TO')) from Book where @rid = b.@rid) as categories
    from Book as b;
2025-07-23 16:19:17,823 [ERROR] ❌ HTTP 500: {'error': 'Error on transaction commit', 'detail': 'com.arcadedb.query.sql.parser.ParseException: Encountered <SELECT> "SELECT" at line 1, column 1. Was expecting one of:     <WHILE> ...     <IF> ...     <FOREACH> ...     ";" ...     ', 'exception': 'com.arcadedb.exception.CommandSQLParsingException'}


{'error': 'Error on transaction commit', 'detail': 'com.arcadedb.query.sql.parser.ParseException: Encountered <SELECT> "SELECT" at line 1, column 1. Was expecting one of:     <WHILE> ...     <IF> ...     <FOREACH> ...     ";" ...     ', 'exception': 'com.arcadedb.exception.CommandSQLParsingException'}


In [48]:
sql = """
-- Get all books with related information
LET books = (
  SELECT 
    title,
    isbn,
    year,
    summary,
    edition,
    (SELECT expand(out('WRITTEN_BY')) FROM Book WHERE @rid = b.@rid) AS authors,
    (SELECT expand(out('PUBLISHED_BY')) FROM Book WHERE @rid = b.@rid) AS publisher,
    (SELECT expand(out('BELONGS_TO')) FROM Book WHERE @rid = b.@rid) AS categories
  FROM Book AS b
);

-- Get all authors with books they wrote
LET authors = (
  SELECT 
    name,
    nationality,
    birth_year,
    (SELECT expand(in('WRITTEN_BY')) FROM Author WHERE @rid = a.@rid) AS books
  FROM Author AS a
);

RETURN [
  { type: 'books', data: $books },
  { type: 'authors', data: $authors }
]

"""

result = run_sql(sql)


2025-07-23 16:09:09,136 [INFO] Running SQL:
-- Get all books with related information
LET books = (
  SELECT 
    title,
    isbn,
    year,
    summary,
    edition,
    (SELECT expand(out('WRITTEN_BY')) FROM Book WHERE @rid = b.@rid) AS authors,
    (SELECT expand(out('PUBLISHED_BY')) FROM Book WHERE @rid = b.@rid) AS publisher,
    (SELECT expand(out('BELONGS_TO')) FROM Book WHERE @rid = b.@rid) AS categories
  FROM Book AS b
);

-- Get all authors with books they wrote
LET authors = (
  SELECT 
    name,
    nationality,
    birth_year,
    (SELECT expand(in('WRITTEN_BY')) FROM Author WHERE @rid = a.@rid) AS books
  FROM Author AS a
);

RETURN [
  { type: 'books', data: $books },
  { type: 'authors', data: $authors }
]
2025-07-23 16:09:09,469 [ERROR] ❌ HTTP 500: {'error': 'Error on transaction commit', 'detail': 'com.arcadedb.query.sql.parser.ParseException: Encountered <LET> "LET" at line 2, column 1. Was expecting one of:     <WHILE> ...     <IF> ...     <FOREACH> ...     ";" ...

In [53]:
script="""
let books = select title, isbn, year, summary, edition from Book;
let authors = select name, nationality, birth_year from Author;

RETURN { 
  "books": books, 
  "authors": authors 
};
"""
result = run_sql(script)

2025-07-23 16:13:34,426 [INFO] Running SQL:
let books = select title, isbn, year, summary, edition from Book;
let authors = select name, nationality, birth_year from Author;

RETURN { 
  "books": books, 
  "authors": authors 
};
2025-07-23 16:13:34,459 [INFO] ✅ Success: {'result': [{'value': {'books': None, 'authors': None}}]}


In [54]:
result

{'result': [{'value': {'books': None, 'authors': None}}]}

In [52]:
run_sql("select title from Book")

2025-07-23 16:13:11,853 [INFO] Running SQL:
select title from Book
2025-07-23 16:13:11,900 [INFO] ✅ Success: {'result': [{'title': 'A Brief History of Time'}, {'title': 'Sapiens: A Brief History of Humankind'}, {'title': 'Thinking, Fast and Slow'}, {'title': 'To Kill a Mockingbird'}, {'title': '1984'}, {'title': 'The Origin of Species'}]}


{'result': [{'title': 'A Brief History of Time'},
  {'title': 'Sapiens: A Brief History of Humankind'},
  {'title': 'Thinking, Fast and Slow'},
  {'title': 'To Kill a Mockingbird'},
  {'title': '1984'},
  {'title': 'The Origin of Species'}]}

In [57]:
script="""
LET publisher = select from Publisher where name = 'Harper';
LET books = select expand( in('PUBLISHED_BY') ) from $publisher;

RETURN {
  "publisher": "Harper",
  "books": $books
};
"""
run_sql(script)

2025-07-23 16:15:34,750 [INFO] Running SQL:
LET publisher = select from Publisher where name = 'Harper';
LET books = select expand( in('PUBLISHED_BY') ) from $publisher;

RETURN {
  "publisher": "Harper",
  "books": $books
};
2025-07-23 16:15:35,062 [INFO] ✅ Success: {'result': [{'value': {'books': [{'@rid': '#1:4', '@type': 'Book', '@cat': 'v', 'title': 'Sapiens: A Brief History of Humankind', 'isbn': '9780062316097', 'year': 2011, 'summary': 'Explores the history and impact of Homo sapiens.', 'edition': 'First'}, {'@rid': '#1:4', '@type': 'Book', '@cat': 'v', 'title': 'Sapiens: A Brief History of Humankind', 'isbn': '9780062316097', 'year': 2011, 'summary': 'Explores the history and impact of Homo sapiens.', 'edition': 'First'}, {'@rid': '#1:4', '@type': 'Book', '@cat': 'v', 'title': 'Sapiens: A Brief History of Humankind', 'isbn': '9780062316097', 'year': 2011, 'summary': 'Explores the history and impact of Homo sapiens.', 'edition': 'First'}, {'@rid': '#1:4', '@type': 'Book', '@cat

{'result': [{'value': {'books': [{'@rid': '#1:4',
      '@type': 'Book',
      '@cat': 'v',
      'title': 'Sapiens: A Brief History of Humankind',
      'isbn': '9780062316097',
      'year': 2011,
      'summary': 'Explores the history and impact of Homo sapiens.',
      'edition': 'First'},
     {'@rid': '#1:4',
      '@type': 'Book',
      '@cat': 'v',
      'title': 'Sapiens: A Brief History of Humankind',
      'isbn': '9780062316097',
      'year': 2011,
      'summary': 'Explores the history and impact of Homo sapiens.',
      'edition': 'First'},
     {'@rid': '#1:4',
      '@type': 'Book',
      '@cat': 'v',
      'title': 'Sapiens: A Brief History of Humankind',
      'isbn': '9780062316097',
      'year': 2011,
      'summary': 'Explores the history and impact of Homo sapiens.',
      'edition': 'First'},
     {'@rid': '#1:4',
      '@type': 'Book',
      '@cat': 'v',
      'title': 'Sapiens: A Brief History of Humankind',
      'isbn': '9780062316097',
      'year': 2011,


In [63]:
sql = """
LET books = select from Book;
LET enriched = (
  FOR b IN $books
    LET authors = (select expand(out('WRITTEN_BY')) from $b),
        publisher = (select expand(out('PUBLISHED_BY')) from $b),
        categories = (selectexpand EXPAND(out('BELONGS_TO')) from $b)
    RETURN {
      title: $b.title,
      isbn: $b.isbn,
      year: $b.year,
      summary: $b.summary,
      edition: $b.edition,
      authors: $authors,
      publisher: $publisher,
      categories: $categories
    }
);
RETURN $enriched;
"""
run_sql(sql)


2025-07-23 16:21:29,110 [INFO] Running SQL:
LET books = select from Book;
LET enriched = (
  FOR b IN $books
    LET authors = (select expand(out('WRITTEN_BY')) from $b),
        publisher = (select expand(out('PUBLISHED_BY')) from $b),
        categories = (selectexpand EXPAND(out('BELONGS_TO')) from $b)
    RETURN {
      title: $b.title,
      isbn: $b.isbn,
      year: $b.year,
      summary: $b.summary,
      edition: $b.edition,
      authors: $authors,
      publisher: $publisher,
      categories: $categories
    }
);
RETURN $enriched;
2025-07-23 16:21:29,152 [ERROR] ❌ HTTP 500: {'error': 'Error on transaction commit', 'detail': 'com.arcadedb.query.sql.parser.ParseException: Encountered <LET> "LET" at line 2, column 1. Was expecting one of:     <WHILE> ...     <IF> ...     <FOREACH> ...     ";" ...     <IF> ...     <IF> ...     <IF> ...     <IF> ...     <IF> ...     ', 'exception': 'com.arcadedb.exception.CommandSQLParsingException'}


{'error': 'Error on transaction commit',
 'detail': 'com.arcadedb.query.sql.parser.ParseException: Encountered <LET> "LET" at line 2, column 1. Was expecting one of:     <WHILE> ...     <IF> ...     <FOREACH> ...     ";" ...     <IF> ...     <IF> ...     <IF> ...     <IF> ...     <IF> ...     ',
 'exception': 'com.arcadedb.exception.CommandSQLParsingException'}

In [65]:
script = """
LET books = SELECT FROM Book;

LET authors = SELECT FROM Author;
LET publishers = SELECT FROM Publisher;
LET categories = SELECT FROM Category;

RETURN (
  FOR b IN $books
    RETURN {
      title: b.title,
      isbn: b.isbn,
      year: b.year,
      summary: b.summary,
      edition: b.edition,
      authors: (SELECT EXPAND(out('WRITTEN_BY')) FROM $b),
      publisher: (SELECT EXPAND(out('PUBLISHED_BY')) FROM $b),
      categories: (SELECT EXPAND(out('BELONGS_TO')) FROM $b)
    }
);
"""
run_sql(script)


2025-07-23 16:24:35,209 [INFO] Running SQL:
LET books = SELECT FROM Book;

LET authors = SELECT FROM Author;
LET publishers = SELECT FROM Publisher;
LET categories = SELECT FROM Category;

RETURN (
  FOR b IN $books
    RETURN {
      title: b.title,
      isbn: b.isbn,
      year: b.year,
      summary: b.summary,
      edition: b.edition,
      authors: (SELECT EXPAND(out('WRITTEN_BY')) FROM $b),
      publisher: (SELECT EXPAND(out('PUBLISHED_BY')) FROM $b),
      categories: (SELECT EXPAND(out('BELONGS_TO')) FROM $b)
    }
);
2025-07-23 16:24:35,241 [ERROR] ❌ HTTP 500: {'error': 'Error on transaction commit', 'detail': 'com.arcadedb.query.sql.parser.ParseException: Encountered <RETURN> "RETURN" at line 7, column 1. Was expecting one of:     <WHILE> ...     <IF> ...     <FOREACH> ...     ";" ...     ', 'exception': 'com.arcadedb.exception.CommandSQLParsingException'}


{'error': 'Error on transaction commit',
 'detail': 'com.arcadedb.query.sql.parser.ParseException: Encountered <RETURN> "RETURN" at line 7, column 1. Was expecting one of:     <WHILE> ...     <IF> ...     <FOREACH> ...     ";" ...     ',
 'exception': 'com.arcadedb.exception.CommandSQLParsingException'}

In [66]:
sql = """
SELECT 
  title,
  isbn,
  year,
  summary,
  edition,
  out('WRITTEN_BY') as authors,
  out('PUBLISHED_BY') as publisher,
  out('BELONGS_TO') as categories
FROM Book
"""
run_sql(sql)


2025-07-23 16:25:52,801 [INFO] Running SQL:
SELECT 
  title,
  isbn,
  year,
  summary,
  edition,
  out('WRITTEN_BY') as authors,
  out('PUBLISHED_BY') as publisher,
  out('BELONGS_TO') as categories
FROM Book
2025-07-23 16:25:53,233 [INFO] ✅ Success: {'result': [{'title': 'A Brief History of Time', 'isbn': '9780553380163', 'year': 1988, 'summary': 'A landmark volume in science writing by one of the great minds of our time.', 'edition': 'First', 'authors': [{'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942}, {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942}, {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942}, {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942}, {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942}, {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942}, {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942}, {'name': 'St

{'result': [{'title': 'A Brief History of Time',
   'isbn': '9780553380163',
   'year': 1988,
   'summary': 'A landmark volume in science writing by one of the great minds of our time.',
   'edition': 'First',
   'authors': [{'name': 'Stephen Hawking',
     'nationality': 'British',
     'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 'birth_year': 1942},
    {'name': 'Stephen Hawking', 'nationality': 'British', 

In [69]:
sql = """
LET authors = SELECT FROM Author WHERE nationality = 'India';
LET books = SELECT EXPAND( in('WRITTEN_BY') ) FROM $authors;

IF $books.size() = 0 THEN
  RETURN "No books";
ELSE
  RETURN $books;
END;
"""
result = run_sql(sql)

2025-07-23 16:27:39,178 [INFO] Running SQL:
LET authors = SELECT FROM Author WHERE nationality = 'India';
LET books = SELECT EXPAND( in('WRITTEN_BY') ) FROM $authors;

IF $books.size() = 0 THEN
  RETURN "No books";
ELSE
  RETURN $books;
END;
2025-07-23 16:27:39,210 [ERROR] ❌ HTTP 500: {'error': 'Error on transaction commit', 'detail': 'com.arcadedb.query.sql.parser.ParseException: Encountered <IDENTIFIER> "$books" at line 4, column 4. Was expecting one of:     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...     "(" ...

In [81]:
def run_cypher(cypher_query):
    url = f"{BASE_URL}/api/v1/command/{DB_NAME}"
    payload = {
        "command": cypher_query,
        "language":"cypher"
    }
    response = requests.post(url, auth=AUTH, json = payload)
    return response.text
# Example Cypher query to get books published by 'Harper'
cypher = """
MATCH (b:Book)-[:PUBLISHED_BY]->(p:Publisher)
WHERE p.name = 'Harper'
RETURN b.title, b.year, p.name AS publisher;
"""

result = run_cypher(cypher)

In [82]:
print(result)

{"result":[{"b.title":"Sapiens: A Brief History of Humankind","b.year":2011,"publisher":"Harper"},{"b.title":"Sapiens: A Brief History of Humankind","b.year":2011,"publisher":"Harper"},{"b.title":"Sapiens: A Brief History of Humankind","b.year":2011,"publisher":"Harper"},{"b.title":"Sapiens: A Brief History of Humankind","b.year":2011,"publisher":"Harper"},{"b.title":"Sapiens: A Brief History of Humankind","b.year":2011,"publisher":"Harper"},{"b.title":"Sapiens: A Brief History of Humankind","b.year":2011,"publisher":"Harper"}]}


In [83]:

def run_cypher(cypher_query):
    url = f"{BASE_URL}/api/v1/command/{DB_NAME}"
    payload = {
        "command": cypher_query,
        "language": "cypher"
    }
    response = requests.post(url, auth=AUTH, json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        return None

# Step 1: Create schema (if not already exists)
def create_schema():
    print("📦 Creating schema...")
    schema_queries = [
        "CREATE VERTEX TYPE Author IF NOT EXISTS;",
        "CREATE VERTEX TYPE Book IF NOT EXISTS;",
        "CREATE VERTEX TYPE Publisher IF NOT EXISTS;",
        "CREATE EDGE TYPE WRITTEN_BY IF NOT EXISTS;",
        "CREATE EDGE TYPE PUBLISHED_BY IF NOT EXISTS;"
    ]
    for query in schema_queries:
        run_cypher(query)
    print("✅ Schema created.\n")

# Step 2: Insert sample data
def insert_sample_data():
    print("✍️ Inserting sample data...")

    # Insert Publisher
    run_cypher("CREATE (p:Publisher {name: 'Harper'})")

    # Insert Authors
    run_cypher("CREATE (:Author {name: 'R.K. Narayan', nationality: 'India'})")
    run_cypher("CREATE (:Author {name: 'George Orwell', nationality: 'UK'})")

    # Insert Books and relationships
    run_cypher("""
    MATCH (a:Author {name: 'R.K. Narayan'}), (p:Publisher {name: 'Harper'})
    CREATE (b:Book {title: 'Malgudi Days', year: 1943})-[:WRITTEN_BY]->(a),
           (b)-[:PUBLISHED_BY]->(p)
    """)

    run_cypher("""
    MATCH (a:Author {name: 'George Orwell'}), (p:Publisher {name: 'Harper'})
    CREATE (b:Book {title: '1984', year: 1949})-[:WRITTEN_BY]->(a),
           (b)-[:PUBLISHED_BY]->(p)
    """)
    
    print("✅ Sample data inserted.\n")

# Step 3: Query example — Get all books published by Harper
def query_books_by_publisher(publisher_name):
    print(f"🔍 Books published by {publisher_name}:\n")
    cypher = f"""
    MATCH (b:Book)-[:PUBLISHED_BY]->(p:Publisher)
    WHERE p.name = '{publisher_name}'
    RETURN b.title AS title, b.year AS year, p.name AS publisher
    """
    result = run_cypher(cypher)
    if result and "result" in result:
        for row in result["result"]:
            print(f"📘 {row['title']} ({row['year']}) - Publisher: {row['publisher']}")
    else:
        print("⚠️ No books found.")

# Step 4: Query example — Books written by Indian authors
def query_books_by_indian_authors():
    print("\n🔍 Books written by Indian authors:\n")
    cypher = """
    MATCH (b:Book)-[:WRITTEN_BY]->(a:Author)
    WHERE a.nationality = 'India'
    RETURN b.title AS title, a.name AS author
    """
    result = run_cypher(cypher)
    if result and "result" in result and result["result"]:
        for row in result["result"]:
            print(f"🇮🇳 {row['title']} - Author: {row['author']}")
    else:
        print("⚠️ No books by Indian authors.")

# Run everything
if __name__ == "__main__":
    create_schema()
    insert_sample_data()
    query_books_by_publisher("Harper")
    query_books_by_indian_authors()

📦 Creating schema...
❌ Error 500: {"error":"Error on transaction commit","detail":"Error on executing Cypher query","exception":"com.arcadedb.exception.CommandParsingException"}
❌ Error 500: {"error":"Error on transaction commit","detail":"Error on executing Cypher query","exception":"com.arcadedb.exception.CommandParsingException"}
❌ Error 500: {"error":"Error on transaction commit","detail":"Error on executing Cypher query","exception":"com.arcadedb.exception.CommandParsingException"}
❌ Error 500: {"error":"Error on transaction commit","detail":"Error on executing Cypher query","exception":"com.arcadedb.exception.CommandParsingException"}
❌ Error 500: {"error":"Error on transaction commit","detail":"Error on executing Cypher query","exception":"com.arcadedb.exception.CommandParsingException"}
✅ Schema created.

✍️ Inserting sample data...
✅ Sample data inserted.

🔍 Books published by Harper:

📘 Malgudi Days (1943) - Publisher: Harper
📘 Malgudi Days (1943) - Publisher: Harper
📘 Malgud

In [84]:
cypher = """
MATCH (b:Book)-[:WRITTEN_BY]->(a:Author),
      (b)-[:PUBLISHED_BY]->(p:Publisher)
WHERE a.nationality = 'India' AND p.name = 'Harper'
RETURN b.title AS title, b.year AS year, a.name AS author, p.name AS publisher
"""

run_cypher(cypher)


{'result': [{'title': 'Malgudi Days',
   'year': 1943,
   'author': 'R.K. Narayan',
   'publisher': 'Harper'},
  {'title': 'Malgudi Days',
   'year': 1943,
   'author': 'R.K. Narayan',
   'publisher': 'Harper'},
  {'title': 'Malgudi Days',
   'year': 1943,
   'author': 'R.K. Narayan',
   'publisher': 'Harper'},
  {'title': 'Malgudi Days',
   'year': 1943,
   'author': 'R.K. Narayan',
   'publisher': 'Harper'}]}